In [1]:
#!pip show tensorflow_datasets
!pip install emnist
!pip install tensorflow_datasets
!pip install opencv_python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 15.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 25.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 34.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 37.7 MB/s eta 0:00:00
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21486 sha256=e40338779b7ff15c76cbedeedbd63c00622bc1f0da4a473e9b57ee36d05664bd
  Stored in directory: /home/jovyan/.cache/pip/wheels/76/40/54/417a4d64a01b61b247658d83597e1dc83c3de01fc0cef44972
Successfully built promise
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 25.0 MB/s eta 0:00:0000:0100:01


In [28]:
import emnist
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow_datasets as tfds
import tensorflow.keras as keras
import sklearn
from sklearn.decomposition import PCA 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
import cv2
from tensorflow.keras.layers import Input, Lambda, Conv2D, MaxPooling2D, BatchNormalization, Dense, Flatten, Activation, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import backend as K
%matplotlib inline

In [34]:
(ds_train, ds_test), info = tfds.load('omniglot', split=['small1', 'test'], with_info=True)

In [35]:
df_train = tfds.as_dataframe(ds_train, info)
df_test  = tfds.as_dataframe(ds_test, info)

In [36]:
train_images = np.stack(df_train['image'])
train_images.shape

(2720, 105, 105, 3)

In [37]:
df_train.columns

Index(['alphabet', 'alphabet_char_id', 'image', 'label'], dtype='object')

In [84]:
len(df_train['label'].unique())

136

In [8]:
#df_train[['alphabet_char_id', 'label']].loc[np.where((df_train['alphabet'] == 27) & (df_train['alphabet_char_id'] == 23))]

In [22]:
''' Data handling general functions '''

def separate_fewshot(test_images, test_labels, n=1):
    oneshot_data = []
    classify_data = []
    for label in np.unique(test_labels):
        for num in np.random.choice(np.where(test_labels == label)[0], n, False):
            oneshot_data.append(num)
    temp = set(oneshot_data)
    for i in range(len(test_labels)):
        if not i in temp: classify_data.append(i)
    oneshot_images = test_images[oneshot_data]
    oneshot_labels = test_labels[oneshot_data]
    classify_images = test_images[classify_data]
    classify_labels = test_labels[classify_data]
    return oneshot_images, oneshot_labels, classify_images, classify_labels

In [23]:
def resize_images(images, size):
    resized_images = []
    for img in images:
        resized_image = cv2.resize(img, (56, 56))
        resized_images.append(resized_image)
    return np.array(resized_images)

In [11]:
def parse_omniglot_dataframe(df, img_size = 56):
    images = resize_images(df['image'], img_size)
    images = images.reshape(-1, img_size * img_size * 3)
    labels = df['label'].to_numpy()
    return (images, labels)

In [12]:
img, lbl = parse_omniglot_dataframe(df_test)
img.shape

(13180, 9408)

In [13]:
def test_PCA(df_train, df_test, n=1, n_components = 32, verbose=False, train=1):
    
    (train_images, train_labels) = parse_omniglot_dataframe(df_train)
    (test_images, test_labels) = parse_omniglot_dataframe(df_test)
    t_alphabets = df_test['alphabet'].to_numpy()
    
    if verbose: print("======= PCA method: Training and evaluating ... =======")
    if verbose: print("Learning background ...")
    pca = PCA(n_components=n_components)
    pca.fit(X=train_images)
    
    matches = 0
    total = 0
    
    if verbose: print("Vectorizing ...")
    for alphabet in np.unique(t_alphabets):
        ind_alphabet = np.where(t_alphabets == alphabet)[0]
        labels = test_labels[ind_alphabet]
        images = test_images[ind_alphabet]
        os_img, os_label, clas_img, clas_label = separate_fewshot(images, labels, n=n)
        
        
        os_img = pca.transform(os_img)
        clas_img = pca.transform(clas_img)

        #if verbose: print("Learning oneshot ...")
        nn = min(train, 5)
        neigh = KNeighborsClassifier(n_neighbors = nn)
        neigh.fit(os_img, os_label)

        #if verbose: print("Predicting ...")
        pred = neigh.predict(clas_img)
        
        matches += np.sum(pred == clas_label)
        total += len(clas_label)
        
    if verbose:
        print("Accuracy: ", matches/total)
        print("======= PCA method: Finished =======")

    return matches/total

In [14]:
# test_PCA(df_train, df_test, verbose=True)

======= PCA method: Training and evaluating ... =======
Learning background ...
Vectorizing ...
Accuracy:  0.16500279530388948
======= PCA method: Finished =======


0.16500279530388948

In [15]:
def test_LDA(df_train, df_test, n=1, n_components = 32, verbose=False, train=1, c=3):
    
    (train_images, train_labels) = parse_omniglot_dataframe(df_train)
    (test_images, test_labels) = parse_omniglot_dataframe(df_test)
    t_alphabets = df_test['alphabet'].to_numpy()
    
    # unique_labels = df_train['label'].unique()
    # subsample_index = []
    # for label in unique_labels:
    #     for ind in np.random.choice(np.where(df_train['label'] == label)[0], c, False):
    #         subsample_index.append(ind)
    # subsample_index = np.array(subsample_index)
    # train_images = train_images[subsample_index]
    # train_labels = train_labels[subsample_index]
    
    if verbose: print("======= LDA method: Training and evaluating ... =======")
    if verbose: print("Learning background ...")
    lda = LDA(n_components=n_components)
    lda.fit(X=train_images,y=train_labels)
    
    matches = 0
    total = 0
    
    if verbose: print("Vectorizing ...")
    for alphabet in np.unique(t_alphabets):
        ind_alphabet = np.where(t_alphabets == alphabet)[0]
        labels = test_labels[ind_alphabet]
        images = test_images[ind_alphabet]
        os_img, os_label, clas_img, clas_label = separate_fewshot(images, labels, n=n)
        
        
        os_img = lda.transform(os_img)
        clas_img = lda.transform(clas_img)

        #if verbose: print("Learning oneshot ...")
        nn = min(train, 5)
        neigh = KNeighborsClassifier(n_neighbors = nn)
        neigh.fit(os_img, os_label)

        #if verbose: print("Predicting ...")
        pred = neigh.predict(clas_img)
        
        matches += np.sum(pred == clas_label)
        total += len(clas_label)
        
    if verbose:
        print("Accuracy: ", matches/total)
        print("======= LDA method: Finished =======")

    return matches/total

In [16]:
# test_LDA(df_train, df_test, verbose=True,c=6)

======= LDA method: Training and evaluating ... =======
Learning background ...
Vectorizing ...
Accuracy:  0.04616244708889066
======= LDA method: Finished =======


0.04616244708889066

In [13]:
def nonlinear_autoencoder(input_size, code_size: int):
    """
    Instanciate and compiles an autoencoder, returns both the autoencoder and just the encoder
    """
    encoder = keras.Sequential([
        keras.layers.Dense(input_size//4, activation='ReLU'),
        keras.layers.Dense(code_size),
    ])
    
    decoder = keras.Sequential([
        keras.layers.Dense(input_size//4, activation='ReLU'),
        keras.layers.Dense(input_size),
    ])
    
    inputs = keras.Input(shape=(input_size,))
    outputs = decoder(encoder(inputs))
    autoencoder = keras.Model(inputs=inputs, outputs=outputs)
    
    autoencoder.compile(optimizer='Adam', loss='MSE')
    return autoencoder, encoder

def linear_autoencoder(input_size, code_size: int):
    """
    Instanciate and compiles an autoencoder, returns both the autoencoder and just the encoder
    """
    encoder = keras.Sequential([
        keras.layers.Dense(code_size),
    ])
    
    decoder = keras.Sequential([
        keras.layers.Dense(input_size),
    ])
    
    inputs = keras.Input(shape=(input_size,))
    outputs = decoder(encoder(inputs))
    autoencoder = keras.Model(inputs=inputs, outputs=outputs)
    
    autoencoder.compile(optimizer='Adam', loss='MSE')
    return autoencoder, encoder

In [14]:
def test_autoencoder(df_train, df_test, autoencoder, img_size=56, num_components=32, n=1, verbose=False, train=1):
    
    (train_images, train_labels) = parse_omniglot_dataframe(df_train, img_size)
    (test_images, test_labels) = parse_omniglot_dataframe(df_test, img_size)
    t_alphabets = df_test['alphabet'].to_numpy()
        
    if verbose: print("======= NL Autoencoder method: Training and evaluating ... =======")
    if verbose: print("Learning background ...")
    autoencoder, encoder = autoencoder(img_size * img_size * 3, code_size=num_components)
    autoencoder.fit(x=train_images,y=train_images, epochs=50, batch_size=64)
    
    matches = 0
    total = 0
    
    if verbose: print("Vectorizing ...")
    for alphabet in np.unique(t_alphabets):
        ind_alphabet = np.where(t_alphabets == alphabet)[0]
        labels = test_labels[ind_alphabet]
        images = test_images[ind_alphabet]
        os_img, os_label, clas_img, clas_label = separate_fewshot(images, labels, n=n)
        
        os_img = encoder.predict(os_img)
        clas_img = encoder.predict(clas_img)

        #if verbose: print("Learning oneshot ...")
        nn = min(train, 5)
        neigh = KNeighborsClassifier(n_neighbors = nn)
        neigh.fit(os_img, os_label)

        #if verbose: print("Predicting ...")
        pred = neigh.predict(clas_img)
        
        matches += np.sum(pred == clas_label)
        total += len(clas_label)
        
    if verbose:
        print("Accuracy: ", matches/total)
        print("======= NL Autoencoder method: Finished =======")

    return matches/total

In [15]:
test_autoencoder(df_train, df_test, autoencoder=nonlinear_autoencoder, verbose=True)

======= NL Autoencoder method: Training and evaluating ... =======
Learning background ...
Epoch 1/50
16/43 [==========>...................] - ETA: 5s - loss: 60162.9414

KeyboardInterrupt: 

In [58]:
def get_siamese_net_and_encoder(input_shape, code_size = 0):
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    encoder = Sequential()
    encoder.add(Conv2D(16, (3, 3), input_shape=input_shape, activation='relu', kernel_regularizer='l2'))
    encoder.add(BatchNormalization())
    encoder.add(Activation('relu'))
    encoder.add(MaxPooling2D(pool_size=2, strides=(2, 2)))
    # encoder.add(Dropout(0.25))
    
    encoder.add(Conv2D(32, (3, 3), kernel_regularizer='l2'))
    encoder.add(BatchNormalization())
    encoder.add(Activation('relu'))
    encoder.add(MaxPooling2D(pool_size=2, strides=(2, 2)))
    # encoder.add(Dropout(0.25))
    
    encoder.add(Flatten())
    
    encoder.add(Dense(32, activation='sigmoid', kernel_regularizer='l2'))
    
    left_emb = encoder(left_input)
    right_emb = encoder(right_input)
    
    L1_Layer = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
    L1_Dist = L1_Layer([left_emb,right_emb])
    OP = Dense(1, activation='sigmoid', kernel_regularizer='l2')(L1_Dist)
    
    siamese_net = Model(inputs=[left_input, right_input], outputs=OP)
    
    return siamese_net, encoder

In [106]:
num_iterations = 5000
batch_size = 64
evaluateEvery = 10

(train_images, train_labels) = resize_images(df_train['image'], 56), df_train['label'].to_numpy()
t_alphabets = df_test['alphabet'].to_numpy()

_, w, h, c = train_images.shape

siamese_net, encoder = get_siamese_net_and_encoder((w, h, c))

siamese_net.compile(
    loss='binary_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)

siamese_net.summary()

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_30 (InputLayer)          [(None, 56, 56, 3)]  0           []                               
                                                                                                  
 input_31 (InputLayer)          [(None, 56, 56, 3)]  0           []                               
                                                                                                  
 sequential_16 (Sequential)     (None, 32)           152768      ['input_30[0][0]',               
                                                                  'input_31[0][0]']               
                                                                                                  
 lambda_14 (Lambda)             (None, 32)           0           ['sequential_16[0][0]',   

In [107]:
def get_image_by_label(label, train_images, train_labels):
    return train_images[np.random.choice(np.where(train_labels == label)[0], 1, False)[0]]

def get_train_data(size, train_images, train_labels, img_size):
    targets = np.zeros((size,))
    targets[size // 2:] = 1
    pairs = [np.zeros((size, img_size, img_size, 3)) for _ in range(2)]
    labels = np.unique(train_labels)
    for i in range(size):
        class1 = np.random.choice(labels, 1)[0]
        class2 = class1
        if i < size // 2:
            while class2 == class1:
                class2 = np.random.choice(labels, 1)[0]
        pairs[0][i] = get_image_by_label(class1, train_images, train_labels)
        pairs[1][i] = get_image_by_label(class2, train_images, train_labels)
    return pairs, targets

In [108]:
for i in range(1, num_iterations + 1):
    x, y = get_train_data(batch_size, train_images, train_labels, 56)
    loss = siamese_net.train_on_batch(x, y)
    if i % evaluateEvery == 0:
        print('Iteration', i, '- Loss:',loss[0],'- Acc:', round(loss[1], 2))

Iteration 10 - Loss: 1.5072311162948608 - Acc: 0.55
Iteration 20 - Loss: 1.3206431865692139 - Acc: 0.64
Iteration 30 - Loss: 1.2171999216079712 - Acc: 0.64
Iteration 40 - Loss: 1.1695829629898071 - Acc: 0.62
Iteration 50 - Loss: 1.1290456056594849 - Acc: 0.62
Iteration 60 - Loss: 0.9871949553489685 - Acc: 0.77
Iteration 70 - Loss: 0.9273760914802551 - Acc: 0.73
Iteration 80 - Loss: 0.9421278834342957 - Acc: 0.69
Iteration 90 - Loss: 0.9149353504180908 - Acc: 0.67
Iteration 100 - Loss: 0.8752526044845581 - Acc: 0.66
Iteration 110 - Loss: 0.8666126728057861 - Acc: 0.73
Iteration 120 - Loss: 0.7953538298606873 - Acc: 0.72
Iteration 130 - Loss: 0.8023759722709656 - Acc: 0.75
Iteration 140 - Loss: 0.7387526035308838 - Acc: 0.78
Iteration 150 - Loss: 0.7180373072624207 - Acc: 0.78
Iteration 160 - Loss: 0.7810739874839783 - Acc: 0.69
Iteration 170 - Loss: 0.6877657175064087 - Acc: 0.77
Iteration 180 - Loss: 0.6942127346992493 - Acc: 0.78
Iteration 190 - Loss: 0.697600781917572 - Acc: 0.86
Ite

In [110]:
matches = 0
total = 0
(test_images, test_labels) = resize_images(df_test['image'], 56), df_test['label'].to_numpy()

print("Vectorizing ...")
for alphabet in np.unique(df_test['alphabet']):
    ind_alphabet = np.where(df_test['alphabet'] == alphabet)[0]
    labels = test_labels[ind_alphabet]
    images = test_images[ind_alphabet]
    os_img, os_label, clas_img, clas_label = separate_fewshot(images, labels, n=1)

    os_img = encoder.predict(os_img)
    clas_img = encoder.predict(clas_img)

    #if verbose: print("Learning oneshot ...")
    #nn = min(train, 5)
    neigh = KNeighborsClassifier(n_neighbors = 1)
    neigh.fit(os_img, os_label)

    #if verbose: print("Predicting ...")
    pred = neigh.predict(clas_img)

    matches += np.sum(pred == clas_label)
    total += len(clas_label)


print("Accuracy: ", matches/total)
print("======= NL Autoencoder method: Finished =======")

Vectorizing ...
16/16 [==============================] - 0s 4ms/step
Accuracy:  0.2534142640364188
======= NL Autoencoder method: Finished =======
